<a href="https://colab.research.google.com/github/ManelSoengas/Pytorch/blob/main/Pytorch_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Treballan amb dades**

---

**torchvision** és una biblioteca complementària de PyTorch pensada per a treballar amb imatges. Conté eines per:

1. Carregar datasets d’imatges reals.

1. Fer transformacions (canvis de mida, rotacions, tensorització...).

1. Carregar models preentrenats per a tasques com classificació o detecció d’objectes.

També existeixen biblioteques semblants per a:

1. torchtext → dades de text (NLP)

1. torchaudio → dades d’àudio (parla, sons)

**Què és torchvision.datasets?**

El mòdul torchvision.datasets conté datasets preparats per a ús directe, com:

**Dataset	:**

1. CIFAR10:	Imatges de 10 classes (cotxe, gos, avió...)
1. COCO:	Imatges amb anotacions d’objectes
1. FashionMNIST:	Roba: sabates, samarretes, pantalons...

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:03<00:00, 7.81MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 140kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.45MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.3MB/s]


**DataLoader** és un wrapper (embolcall) que s'encarrega d'agafar un Dataset i transformar-lo en lots (batches) per entrenar o validar el model. És com un repartidor eficient de dades.

**Funcions clau del DataLoader:**

1. Iteració automàtica sobre el dataset

1. Agrupació en batches (per entrenar en paral·lel)

1. Mescla aleatòria de dades (shuffle)

1. Càrrega paral·lela amb múltiples processos (num_workers)

1. Mostratge personalitzat (amb sampler)


```
print(f"Shape of X [N, C, H, W]: {X.shape}")

Mostra la forma del tensor X.

X.shape serà una tupla com (64, 1, 28, 28) per FashionMNIST.

N: nombre d'exemples al lot (batch size)

C: canals (1 per imatges en blanc i negre)

H, W: alçada i amplada (28x28 píxels)
```



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# **Creació de models**

---
Com definir una xarxa neuronal en PyTorch i com optimitzar-la per al seu funcionament en diferents dispositius (CPU o GPU).




In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# **Optimitzant el model**

---



In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.292857  [   64/60000]
loss: 2.288896  [ 6464/60000]
loss: 2.264583  [12864/60000]
loss: 2.265701  [19264/60000]
loss: 2.232326  [25664/60000]
loss: 2.208551  [32064/60000]
loss: 2.214118  [38464/60000]
loss: 2.174933  [44864/60000]
loss: 2.176874  [51264/60000]
loss: 2.147728  [57664/60000]
Test Error: 
 Accuracy: 51.4%, Avg loss: 2.135032 

Epoch 2
-------------------------------
loss: 2.140906  [   64/60000]
loss: 2.137555  [ 6464/60000]
loss: 2.074248  [12864/60000]
loss: 2.103456  [19264/60000]
loss: 2.031386  [25664/60000]
loss: 1.979567  [32064/60000]
loss: 2.007549  [38464/60000]
loss: 1.920470  [44864/60000]
loss: 1.931703  [51264/60000]
loss: 1.872960  [57664/60000]
Test Error: 
 Accuracy: 58.1%, Avg loss: 1.855871 

Epoch 3
-------------------------------
loss: 1.881663  [   64/60000]
loss: 1.861124  [ 6464/60000]
loss: 1.736813  [12864/60000]
loss: 1.793527  [19264/60000]
loss: 1.663408  [25664/60000]
loss: 1.629219  [32064/600

# **Guardant el model**

---



In [11]:
torch.save(model.state_dict(), "/content/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/content/model.pth", weights_only=True))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
